```
This notebook sets up and runs a test case for analyzing Kelvin waves
Copyright (C) 2016, 2017, 2018  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
import scipy as scipy
from scipy import integrate
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#%setup_logging --out galewsky.log
%cuda_context_handler gpu_ctx

# Kelvin waves!
Defining and analysing a test case for Kelvin waves

In [ ]:
# DEFINE PARAMETERS

sim_args = {
    "gpu_ctx": gpu_ctx,
    "nx": 500, "ny": 200,
    "dx": 10000.0, "dy": 10000.0,
    "dt": 20,
    "g": 9.81,
    "f": 1.2e-4,
    "coriolis_beta": 0.0,
    "r": 0.0,
    "boundary_conditions": Common.BoundaryConditions(1,2,1,2)
}

cdklm_args = {
    "rk_order": 2,
}

depth = 100
rossby_radius = np.sqrt(sim_args["g"]*depth)/sim_args["f"]
phase_speed = np.sqrt(sim_args['g']*depth)
periode = sim_args['dx']*sim_args['nx']/phase_speed

print("rossby_radius: ", rossby_radius)
print("phase_speed:   ", phase_speed)
print("periode:       ", periode)

In [ ]:
def add_hovmuller(i, t, hovmuller_data, data):
    hovmuller_data['t'][i] = t
    for y, hovmuller in hovmuller_data['hovmuller']:
        hovmuller[i,:] = data[y,:]
    hovmuller_data['mass'][i] = data.sum()

def run_sim(sim, i, sub_dt, hovmuller_data, kill_left=-1):
    if (i>0):
        t = sim.step(sub_dt*sim.dt)
    else:
        t = 0.0

    if i == kill_left:
        eta1, u1, v1 = sim.download()
        eta1[:, :int(np.floor(sim.nx/2))] = 0
        u1[:, :int(np.floor(sim.nx/2))] = 0
        v1[:, :int(np.floor(sim.nx/2))] = 0
        eta_prev, u_prev, v_prev = sim.downloadPrevTimestep()
        eta_prev[:, :int(np.floor(sim.nx/2))] = 0
        u_prev[:, :int(np.floor(sim.nx/2))] = 0
        v_prev[:, :int(np.floor(sim.nx/2))] = 0

        sim.gpu_data.h0.upload(sim.gpu_stream, eta1)
        sim.gpu_data.hu0.upload(sim.gpu_stream, u1)
        sim.gpu_data.hv0.upload(sim.gpu_stream, v1)

        sim.gpu_data.h1.upload(sim.gpu_stream, eta_prev)
        sim.gpu_data.hu1.upload(sim.gpu_stream, u_prev)
        sim.gpu_data.hv1.upload(sim.gpu_stream, v_prev)


    eta1, u1, v1 = sim.download(interior_domain_only=True)

    if hovmuller_data is not None:
        add_hovmuller(i, t, hovmuller_data, eta1)
    return t
        
def sim_animation(sim, T, sub_dt=10, hovmuller_data=None, kill_left=-1):
    """
    Creates an animation of the simulator based on T frames, with frames sub_dt*sim.dt appart.
    
    If hovmuller_data is provided, cross sections of the simulation is stored in the hovmuller_data arrays.
    hovmuller_data is expected to be of the format:
    hovmuller_data = {'t': 1D array of size T which will contain the simulation times,
            'hovmuller': [(y1, 2D array of size T times nx to store the cross sections),
                          (y2, 2D array of size T times nx to store the cross sections),
                           ...]}
    """
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(16, 16))
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    eta_max = eta1.max()*0.5
    hu_max = max(v1.max(), u1.max())
    
    ax_eta = plt.subplot(3,1,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-eta_max, vmax=eta_max, extent=domain_extent)
    #ax_eta.colorbar()
    
    ax_u = plt.subplot(3,1,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_u.colorbar()
    
    ax_v = plt.subplot(3,1,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_v.colorbar()
    
    #Helper function which simulates and plots the solution
    def animate(i):
        
        t = run_sim(sim, i, sub_dt, hovmuller_data, kill_left)
        eta1, u1, v1 = sim.download(interior_domain_only=True)

        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)

        fig.sca(ax_u)
        sp_u.set_data(u1)

        fig.sca(ax_v)
        sp_v.set_data(v1)

        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print(".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

In [ ]:
def eta_cell(r_i, r_j, rossby_radius, coriolis):
    f_func = 1.0 + np.tanh((-r_i + rossby_radius)/(rossby_radius/3))

    if coriolis:
        return np.exp(-r_j/rossby_radius)*f_func
    else:
        return f_func
    
def init_eta(eta, ghost_cells, rossby_radius, coriolis=True):
    ny, nx = eta.shape
    x_0 = nx/2
    y_0 = ghost_cells[2]-0.5
    for j in range(0, ny):
        for i in range(0, nx):
            r_j = np.sqrt((j-y_0)**2)*sim_args["dy"]
            r_i = np.sqrt((i-x_0)**2)*sim_args["dx"]
            
            eta[j,i] = eta_cell(r_i, r_j, rossby_radius, coriolis)
            
            
def init_hu(hu, ghost_cells, rossby_radius, geoconst, coriolis=True):
    if not coriolis:
        return
    ny, nx = hu.shape
    x_0 = nx/2
    y_0 = ghost_cells[2]-0.5
    for j in range(0, ny):
        for i in range(0, nx):
            r_j = np.sqrt((j-y_0)**2)*sim_args["dy"]
            r_i = np.sqrt((i-x_0)**2)*sim_args["dx"]
            
            eta_c = eta_cell(r_i, r_j, rossby_radius, coriolis)
            hu[j,i] = -geoconst*(-1.0/rossby_radius)*np.sign(j-y_0)*eta_c
            
            
    

In [ ]:
%%time
def runPeriodicSims(sims=[1,1,1,1], animate=False, coriolis=True, cut_after=-1):
    if animate:
        assert(sum(sims) == 1), "animation supported for one sim only"
    
    sim_args = {
        "gpu_ctx": gpu_ctx,
        "nx": 500, "ny": 200,
        "dx": 10000.0, "dy": 10000.0,
        "dt": 20,
        "g": 9.81,
        "f": 1.2e-4,
        "coriolis_beta": 0.0,
        "r": 0.0,
        "boundary_conditions": Common.BoundaryConditions(1,2,1,2)
        #"boundary_conditions": Common.BoundaryConditions(2,2,2,2)    
        #"boundary_conditions": Common.BoundaryConditions(1,1,1,1)    
    }
    fvm_args = {
        "rk_order": 2,
        "theta": 1.0
    }
    ctcs_args = {
        "A": 0.0
    }
    depth = 100
    rossby_radius = np.sqrt(sim_args["g"]*depth)/sim_args["f"]
    phase_speed = np.sqrt(sim_args['g']*depth)
    periode = sim_args['dx']*sim_args['nx']/phase_speed
    geoconst = sim_args['g']*depth/sim_args['f']
    
    T = 101
    sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/

    print("rossby_radius: ", rossby_radius)
    print("phase_speed:   ", phase_speed)
    print("periode:       ", periode)
    
    hovmuller_data = {'FBL':   {'t': np.zeros(T),
                                'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                              (30, np.zeros((T, sim_args["nx"])))],
                                'mass': np.zeros(T)},
                      'CTCS':  {'t': np.zeros(T),
                                'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                             (30, np.zeros((T, sim_args["nx"])))],
                                'mass': np.zeros(T)},
                      'KP07':  {'t': np.zeros(T),
                                'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                             (30, np.zeros((T, sim_args["nx"])))],
                                'mass': np.zeros(T)},
                      'CDKLM': {'t': np.zeros(T),
                                'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                              (30, np.zeros((T, sim_args["nx"])))],
                                'mass': np.zeros(T)}
                     }

    if not coriolis:
        sim_args["f"] = 0.0

    for i, sim_name in zip(range(4), ["FBL", "CTCS", "KP07", "CDKLM"]):
        print("")
        if not sims[i]:
            print("skipping ", sim_name)
            continue
        print ("starting ", sim_name)
    
        ghosts = [1,1,1,1]
        if i > 1:
            ghosts = [2,2,2,2]

        dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
                     sim_args["nx"] + ghosts[1]+ghosts[3])

        eta = np.zeros(dataShape, dtype=np.float32, order='C');
        hu, hv, H = None, None, None
        if i == 0:
            hu = np.zeros((dataShape[0], dataShape[1]-1), dtype=np.float32);
            hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  
            H = np.ones(dataShape, dtype=np.float32)*depth
        elif i == 1:
            hu = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
            hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  
            H = np.ones(dataShape, dtype=np.float32)*depth
        else:
            hu = np.zeros(dataShape, dtype=np.float32, order='C');
            hv = np.zeros(dataShape, dtype=np.float32, order='C');
            H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32)*depth

        init_eta(eta, ghosts, rossby_radius, coriolis)
        init_hu( hu,  ghosts, rossby_radius, geoconst, coriolis)
        
        init_args =  {
            "eta0" : eta,
            "hu0"  : hu,
            "hv0"  : hv,
            "H"    : H
        }
        
        sim = None
        if sim_name == "FBL":
            sim = FBL.FBL(**sim_args, **init_args)
        elif sim_name == "CTCS":
            sim = CTCS.CTCS(**sim_args, **init_args, **ctcs_args)
        elif sim_name == "KP07":
            sim = KP07.KP07(**sim_args, **init_args)
        elif sim_name == "CDKLM":
            sim = CDKLM16.CDKLM16(**sim_args, **init_args, **fvm_args)
           
        kill_left = -1
        if not coriolis:
            kill_left = 3
            
        if animate:
            return sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=None, kill_left=kill_left)

        else:
            for t in range(T):
                run_sim(sim, t, sub_dt, hovmuller_data[sim_name], kill_left=kill_left)
                print(".", end='')
                if t == cut_after:
                    break
    return hovmuller_data

In [ ]:
# Check solution after one periode:
def plot_periodic_solution(hovmuller_data, title):
    fig = plt.figure(figsize=(15,5))
    nx = hovmuller_data['hovmuller'][0][1].shape[1]
    x_mid = int(nx/2)
    x_start = x_mid - int(nx/6)
    x_end   = x_mid + int(nx/6)
    for hovmuller in hovmuller_data['hovmuller']:
        plt.plot(hovmuller[1][0 , x_start:x_end], 'b:')
        plt.plot(hovmuller[1][10, x_start:x_end], 'y-.')
        plt.plot(hovmuller[1][50, x_start:x_end], 'r--')
        plt.plot(hovmuller[1][-1, x_start:x_end], 'g-')
        plt.title(title)
    plt.grid()
    print('final t/10: ', hovmuller_data['t'][-1]/10)
    print('half  t/5:  ', hovmuller_data['t'][50]/5)
    print('one p t:    ', hovmuller_data['t'][10])
    print('periode:    ', periode)
    print('caption: blue = initial data, red = 5 periods, green = 10 periods')
    
def plot_total_mass(all_data):
    fig = plt.figure(figsize=(15,5))
    max_mass = 0.0
    for sim_name in ["FBL", "CTCS", "KP07", "CDKLM"]:
        plt.plot(all_data[sim_name]['t'] / periode, 
                 all_data[sim_name]['mass']-all_data[sim_name]['mass'][0], 
                 label=sim_name)
        max_mass = max(max_mass, np.max(all_data[sim_name]['mass']))
    plt.legend()
    plt.title('Total mass of eta')
    #plt.ylim([0, max_mass*1.1])
    plt.grid()
    print("total mass: ", max_mass)

In [ ]:
%%time
reload(CDKLM16)
coriolis_data = runPeriodicSims(sims=[1,1,1,1])#, cut_after=30)

In [ ]:
plot_periodic_solution(coriolis_data['FBL'], 'FBL')
plot_periodic_solution(coriolis_data['CTCS'], 'CTCS')
plot_periodic_solution(coriolis_data['KP07'], 'KP07')
plot_periodic_solution(coriolis_data['CDKLM'], 'CDKLM')

plot_total_mass(coriolis_data)

In [ ]:
#plot_periodic_solution(coriolis_data['FBL'], 'FBL')
#plot_periodic_solution(coriolis_data['CTCS'], 'CTCS')
plot_periodic_solution(coriolis_data['KP07'], 'KP07')
plot_periodic_solution(coriolis_data['CDKLM'], 'CDKLM')

plot_total_mass(coriolis_data)

In [ ]:
#plot_periodic_solution(coriolis_data['FBL'], 'FBL')
plot_periodic_solution(coriolis_data['CTCS'], 'CTCS')
plot_periodic_solution(coriolis_data['KP07'], 'KP07')
plot_periodic_solution(coriolis_data['CDKLM'], 'CDKLM')

plot_total_mass(coriolis_data)

In [ ]:
crash
noncoriolis_data = runPeriodicSims(sims=[0,0,0,1], coriolis=False)

In [ ]:
plot_periodic_solution(noncoriolis_data['FBL'], 'FBL')
plot_periodic_solution(noncoriolis_data['CTCS'], 'CTCS')
plot_periodic_solution(noncoriolis_data['KP07'], 'KP07')
plot_periodic_solution(noncoriolis_data['CDKLM'], 'CDKLM')
plot_total_mass(noncoriolis_data)

In [ ]:
print(noncorlios_data['CDKLM']['mass'][0]/noncorlios_data['CDKLM']['mass'][-1])

In [ ]:
runPeriodicSims(sims=[0,1,0,0], coriolis=True, animate=True, cut_after=30)

In [ ]:
def init_mid_bump(eta, hv, hu, args):
    geo_const = args['g']*depth/args['f']
    rossby_radius = np.sqrt(args['g']*depth)/args['f']
    ny, nx = eta.shape
    x0 = nx/2
    y0 = ny/2
    r0 = 0.5*rossby_radius #10*args['dx']
    for j in range(ny):
        for i in range(nx):
            r = ((i - x0)*args['dx'])**2 + ((j-y0)*args['dy'])**2
            e = 0.3*np.exp(-r/r0**2)
            eta[j,i] = e
            #hu[j,i] = 2*geo_const*(j-y0)*e/r0
            #hv[j,i] =-2*geo_const*(i-x0)*e/r0

small_args = sim_args
small_args['nx'] = 50
small_args['ny'] = 50
small_args['boundary_conditions'] = Common.BoundaryConditions(1,1,1,1)
#small_args['boundary_conditions'] = Common.BoundaryConditions(1,2,1,2)

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2]) #

dataShape = (small_args["ny"] + ghosts[0]+ghosts[2], 
             small_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (small_args["ny"] + ghosts[0]+ghosts[2]+1, 
             small_args["nx"] + ghosts[1]+ghosts[3]+1)

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hv  = np.zeros(dataShape, dtype=np.float32, order='C');
hu  = np.zeros(dataShape, dtype=np.float32, order='C');

#init_eta(eta, ghosts, rossby_radius)
#init_hu(hu, ghosts, rossby_rad)
init_mid_bump(eta, hv, hu, small_args)





cdklm_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : np.ones(dataShapeHi, dtype=np.float32, order='C')*depth
}

sim.cleanUp()

reload(CDKLM16)
sim = CDKLM16.CDKLM16(**small_args, **cdklm_init_args)

T = 10
sub_dt=100

run_sim(sim, 1, sub_dt*T*10, None)
#sim_animation(sim, T, sub_dt=sub_dt)
#sim_animation(sim, 1)

In [ ]:
eta_fin, hu_fin, hv_fin = sim.download(True)
print (eta[2:-2, 2:-2].sum())
print (eta_fin.sum())
       

In [ ]:
#CTCS
ghosts = [1,1,1,1]
validDomain = np.array([1,1,1,1]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hu = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

center_x = (sim_args["nx"]+2)/2
center_y = 0
y_mid = (sim_args["ny"]+2)/2

H = np.ones(eta.shape, dtype=np.float32, order='C')*depth

def init_eta(eta, ghost_cells):
    ny, nx = eta.shape
    x_0 = nx/2
    y_0 = ghost_cells[2]
    for j in range(0, ny):
        for i in range(0, nx):
            r_j = np.sqrt((j-y_0)**2)*sim_args["dy"]
            r_i = np.sqrt((i-x_0)**2)*sim_args["dx"]
            
            f_func = 1.0 + np.tanh((-r_i + rossby_radius)/(rossby_radius/3))
            #f_func = 1.0 + np.tanh((-r_i + rossby_radius)/(rossby_radius/2))
        
            eta[j,i] = np.exp(-r_j/rossby_radius)*f_func
            #eta[j,i] = f_func
            
init_eta(eta, ghosts)


for j in range(1,sim_args["ny"]+1):
    for i in range(1,sim_args["nx"]+2):
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

#
#plt.plot(eta[2:,250])
#plt.plot(eta[2,:])
#plt.plot(eta[20,:])
#osidhgd

        
ctcs_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : H,
    "A" : 0
}

reload(CTCS)
sim = CTCS.CTCS(**sim_args, **ctcs_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for 10 whole periodes
#sub_dt = 4*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes
#T=11

CTCS_hovmuller = {'t': np.zeros(T),
                  'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                (10, np.zeros((T, sim_args["nx"]))), 
                                (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=CTCS_hovmuller, kill_left=-1)
#sim_animation(sim, 1)

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (sim_args["ny"] + ghosts[0]+ghosts[2]+1, 
             sim_args["nx"] + ghosts[1]+ghosts[3]+1)

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hv  = np.zeros(dataShape, dtype=np.float32, order='C');
hu  = np.zeros(dataShape, dtype=np.float32, order='C');

center_x = (sim_args["nx"]+2)/2
center_y = 0


for j in range(0, sim_args["ny"]+4):
    for i in range(0,sim_args["nx"]+4):
        scale = 1
        #r_i = bump_size*(i-center_x)**2 *sim_args["dx"]**2
        #r_j = bump_size*(j-center_y) *sim_args["dy"]
        
        r = np.sqrt((i-center_x)**2 + (j-center_y)**2)*sim_args["dx"]
        
        #f_func = np.exp(-r_i/rossby_radius)
        
        #eta[j,i] = np.exp(-scale*r/rossby_radius)
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        

init_eta(eta, ghosts)

        
const = sim_args["g"]*depth/sim_args["f"]
for j in range(2,sim_args["ny"]+2):
    for i in range(2,sim_args["nx"]+2):
        #hv[j,i] =  const*(eta[j, i+1] - eta[j,i-1])/(2*sim_args["dx"])
        hu[j,i] = -const*(eta[j+1, i] - eta[j-1,i])/(2*sim_args["dy"])
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

                             
kp07_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : np.ones(dataShapeHi, dtype=np.float32, order='C')*depth
}


sim = KP07.KP07(**sim_args, **cdklm_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
#sub_dt = periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
KP07_hovmuller = {'t': np.zeros(T),
                   'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                 (10, np.zeros((T, sim_args["nx"]))), 
                                 (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=KP07_hovmuller)
#sim_animation(sim, 1)

In [ ]:
#FBL
ghosts = [1,1,1,1]
validDomain = np.array([1,1,1,1]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hu = np.zeros((dataShape[0], dataShape[1]-1), dtype=np.float32);
hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

center_x = (sim_args["nx"]+2)/2
center_y = 0
y_mid = (sim_args["ny"]+2)/2

H = np.ones(eta.shape, dtype=np.float32, order='C')*depth

init_eta(eta, ghosts)


#for j in range(sim_args["ny"]):
#    for i in range(sim_args["nx"]+1):
#        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

#
#plt.plot(eta[2:,250])
#plt.plot(eta[2,:])
#plt.plot(eta[20,:])
#osidhgd

        
fbl_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : H
}

reload(FBL)
sim = FBL.FBL(**sim_args, **fbl_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for ten whole periodes
FBL_hovmuller = {'t': np.zeros(T),
                  'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                (10, np.zeros((T, sim_args["nx"]))), 
                                (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=FBL_hovmuller, kill_left=3)
#sim_animation(sim, 1)

In [ ]:
def plot_hovmuller(hovmuller_data, title):
    domain_extent = [0, sim_args['nx']*sim_args['dx'], 0, hovmuller_data['t'][-1]]
    max_eta = eta.max()
    for hovmuller in hovmuller_data['hovmuller']:

        fig = plt.figure()
        plt.imshow(hovmuller[1], 
                   origin='lower', extent=domain_extent, aspect='auto',
                   interpolation='none', vmin=0, vmax=max_eta,)
        plt.title(title +  " for y = " + str(hovmuller[0]))
        plt.colorbar()
        
plot_hovmuller(CTCS_hovmuller, "CTCS")
plot_hovmuller(CDKLM_hovmuller, "CDKLM")
plot_hovmuller(KP07_hovmuller, "KP07")


In [ ]:


plot_periodic_solution(CTCS_hovmuller, "CTCS")
plot_periodic_solution(FBL_hovmuller, "FBL")
print('A: ', ctcs_init_args['A'])
plot_periodic_solution(CDKLM_hovmuller, "CDKLM")
plot_periodic_solution(KP07_hovmuller, "KP07")
print(sim_args["nx"])

Beskrivelse av init + boundary conditions. Beskrivelse av valg av parametere sammenlignet med Rossbyradien.

Analytisk  fasehastighet, og plot som viser at den stemmer. HovmÃ¼ller diagram med karakteristikkene.

Vurdere Ã¥ si noe om energibetraktninger.
Sjekke at bÃ¸lgen er lik etter en "runde" (en periode). Fasehastighet er $\sqrt{gH}$.
